In [ ]:
# First install system dependencies and Python packages
!sudo apt-get install -y libzbar0
!pip install qrcode[pil] pillow pyzbar

# Now the imports should work
import qrcode
from PIL import Image, ImageDraw, ImageOps, ImageColor
import numpy as np
import matplotlib.pyplot as plt
import random
from pyzbar.pyzbar import decode
import io
import IPython.display
import ipywidgets as widgets
from IPython.display import display

**Basic QR Code Generation**

In [ ]:
def generate_basic_qr(data, fill_color="black", back_color="white", size=10):
    """
    Generate a basic QR code with customizable colors and size

    Args:
        data (str): Data to encode in QR code
        fill_color (str): Color of QR code modules
        back_color (str): Background color
        size (int): Controls the size of QR code (1-40, higher means more data capacity)

    Returns:
        PIL.Image: Generated QR code image
    """
    qr = qrcode.QRCode(
        version=size,
        error_correction=qrcode.constants.ERROR_CORRECT_H,
        box_size=10,
        border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)

    img = qr.make_image(fill_color=fill_color, back_color=back_color)
    return img

# Example usage
basic_qr = generate_basic_qr("Hello World!", fill_color="navy", back_color="lightgray")
basic_qr

**Artistic QR Code with Center Logo**

In [ ]:
def generate_artistic_qr(data, logo_path=None, fill_color="black", back_color="white", size=8):
    """
    Generate QR code with artistic elements and optional center logo

    Args:
        data (str): Data to encode
        logo_path (str): Path to logo image (optional)
        fill_color (str): Main QR color
        back_color (str): Background color
        size (int): QR size parameter

    Returns:
        PIL.Image: Artistic QR code
    """
    # Generate base QR code
    qr = generate_basic_qr(data, fill_color, back_color, size)
    qr = qr.convert("RGBA")

    if logo_path:
        # Open and process logo
        logo = Image.open(logo_path).convert("RGBA")

        # Resize logo to 25% of QR size
        qr_width, qr_height = qr.size
        logo_size = min(qr_width, qr_height) // 4
        logo.thumbnail((logo_size, logo_size), Image.Resampling.LANCZOS)

        # Create circular mask for logo
        mask = Image.new("L", (logo_size, logo_size), 0)
        draw = ImageDraw.Draw(mask)
        draw.ellipse((0, 0, logo_size, logo_size), fill=255)

        # Paste logo in center of QR
        pos = ((qr_width - logo_size) // 2, (qr_height - logo_size) // 2)
        qr.paste(logo, pos, mask)

    return qr

# Example with sample logo (using a simple generated circle as example)
logo_img = Image.new("RGBA", (200, 200), (0, 0, 0, 0))
draw = ImageDraw.Draw(logo_img)
draw.ellipse((50, 50, 150, 150), fill=(255, 0, 0, 128))
logo_img.save("/tmp/logo.png")

artistic_qr = generate_artistic_qr("Artistic QR Code", "/tmp/logo.png",
                                  fill_color="darkgreen", back_color="beige")
artistic_qr

**Gradient Colored QR Code**

In [ ]:
def generate_gradient_qr(data, colors=("red", "blue"), size=8):
    """
    Generate QR code with gradient coloring

    Args:
        data (str): Data to encode
        colors (tuple): Two color names for gradient
        size (int): QR size parameter

    Returns:
        PIL.Image: Gradient colored QR code
    """
    # Generate black/white QR first
    qr = generate_basic_qr(data, "black", "white", size)
    qr = qr.convert("RGB")

    # Create gradient
    width, height = qr.size
    gradient = Image.new("RGB", (width, height))
    draw = ImageDraw.Draw(gradient)

    for y in range(height):
        # Calculate gradient color at this y position
        ratio = y / height
        r = int((1-ratio) * ImageColor.getcolor(colors[0], "RGB")[0] +
                ratio * ImageColor.getcolor(colors[1], "RGB")[0])
        g = int((1-ratio) * ImageColor.getcolor(colors[0], "RGB")[1] +
                ratio * ImageColor.getcolor(colors[1], "RGB")[1])
        b = int((1-ratio) * ImageColor.getcolor(colors[0], "RGB")[2] +
                ratio * ImageColor.getcolor(colors[1], "RGB")[2])

        draw.line([(0, y), (width, y)], fill=(r, g, b))

    # Apply gradient only to black parts (QR modules)
    qr_data = np.array(qr)
    gradient_data = np.array(gradient)

    # Where QR is black (0,0,0), use gradient color
    mask = (qr_data[:,:,0] == 0) & (qr_data[:,:,1] == 0) & (qr_data[:,:,2] == 0)
    qr_data[mask] = gradient_data[mask]

    return Image.fromarray(qr_data)

# Example usage
gradient_qr = generate_gradient_qr("Gradient QR", ("purple", "orange"))
gradient_qr

**QR Code Decoding Function**

In [10]:
def decode_qr(image):
    """
    Decode data from QR code image

    Args:
        image (PIL.Image): Image containing QR code

    Returns:
        str: Decoded data or error message
    """
    try:
        # Convert to grayscale for better decoding
        if image.mode != "L":
            image = image.convert("L")

        # Decode the QR code
        decoded = decode(image)

        if decoded:
            return decoded[0].data.decode("utf-8")
        else:
            return "No QR code found or could not decode"
    except Exception as e:
        return f"Error decoding QR: {str(e)}"

# Test decoding
qr_to_decode = generate_basic_qr("Decode me!")
decode_result = decode_qr(qr_to_decode)
print(f"Decoded content: {decode_result}")

Decoded content: Decode me!


**Interactive Colab Interface**

In [11]:
# @title QR Code Generator { run: "auto" }
from IPython.display import display
import ipywidgets as widgets

# Create interactive widgets
data_input = widgets.Textarea(value="Enter your text here", description="Data:")
qr_type = widgets.Dropdown(options=["Basic", "Artistic", "Gradient"], description="Type:")
color1 = widgets.ColorPicker(description="Color 1", value="black")
color2 = widgets.ColorPicker(description="Color 2", value="white")
size_slider = widgets.IntSlider(min=1, max=10, value=5, description="Size:")
generate_btn = widgets.Button(description="Generate QR Code")
output = widgets.Output()

# File upload for artistic QR
upload = widgets.FileUpload(accept="image/*", multiple=False)

def on_generate_clicked(b):
    with output:
        output.clear_output()
        data = data_input.value

        if qr_type.value == "Basic":
            img = generate_basic_qr(data, color1.value, color2.value, size_slider.value)
            display(img)

        elif qr_type.value == "Artistic":
            if upload.value:
                # Save uploaded image to temp file
                uploaded_file = next(iter(upload.value))
                with open("/tmp/uploaded_logo", "wb") as f:
                    f.write(uploaded_file["content"])
                img = generate_artistic_qr(data, "/tmp/uploaded_logo",
                                         color1.value, color2.value, size_slider.value)
            else:
                img = generate_artistic_qr(data, None, color1.value, color2.value, size_slider.value)
            display(img)

        elif qr_type.value == "Gradient":
            img = generate_gradient_qr(data, (color1.value, color2.value), size_slider.value)
            display(img)

        # Show decode result
        decoded = decode_qr(img)
        print(f"Decoded content: {decoded}")

generate_btn.on_click(on_generate_clicked)

# Display the UI
display(widgets.VBox([
    data_input,
    qr_type,
    color1,
    color2,
    size_slider,
    upload,
    generate_btn,
    output
]))